In [1]:
from ferc_xbrl_extractor.cli import TAXONOMY_MAP
import pandas as pd
from ferc_xbrl_extractor.xbrl import get_fact_tables
from pathlib import Path
from stringcase import snakecase
from collections import Counter
import itertools

In [2]:
df = pd.read_pickle("../lost_facts.pickle")

tmp_path="./"
tables = get_fact_tables(
    taxonomy_path=TAXONOMY_MAP[1],
    form_number=1,
    db_path="path",
    metadata_path=Path(tmp_path) / "metadata.json",
)

In [7]:
Counter(df.entity)

Counter({'C000537': 3990,
         'C000746': 3150,
         'C000532': 2574,
         'C007667': 2550,
         'C000530': 2460,
         'C000041': 2350,
         'C000536': 2160,
         'C001009': 2106,
         'C002196': 1908,
         'C001555': 1885,
         'C000620': 1692,
         'C004995': 1680,
         'C008999': 1664,
         'C000535': 1583,
         'C000533': 1545,
         'C001016': 1346,
         'C000538': 1304,
         'C003184': 1216,
         'C000135': 1135,
         'C001646': 1118,
         'C000136': 1098,
         'C000534': 1076,
         'C001789': 1064,
         'C000913': 1023,
         'C007565': 994,
         'C001436': 962,
         'C000744': 957,
         'C000317': 952,
         'C001132': 918,
         'C000379': 880,
         'C000825': 848,
         'C001218': 847,
         'C000851': 837,
         'C000318': 827,
         'C001257': 813,
         'C000312': 809,
         'C000905': 809,
         'C008998': 780,
         'C000316': 764,
 

In [3]:
idx=df.table_candidates.apply(lambda x: any(["utility_type_axis" in tables[name].axes for name in x]))
missing_utl_type = df[idx]

print(f"Percent lost facts missing utl type axis: {len(missing_utl_type)/len(df)}")

Percent lost facts missing utl type axis: 0.8633052612211787


In [17]:
print(len(df))

119573


In [5]:
# Look for facts that are only missing the utility type axis and limit to those tables
refined_candidates = []
for idx, row in missing_utl_type.iterrows():
    dims = set([snakecase(name) for name, _ in row.dimensions])
    refined_candidates.append([
        name for name in row.table_candidates
        if (set(tables[name].axes) - dims) == set(["utility_type_axis"])
        and set(tables[name].axes) > dims
    ])

missing_utl_type["refined_candidates"] = refined_candidates

/tmp/ipykernel_30485/4286940243.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missing_utl_type["refined_candidates"] = refined_candidates


In [10]:
sum(missing_utl_type.refined_candidates.apply(lambda x: len(x) == 1)) / len(df)

0.8029739155160446

In [13]:
Counter(itertools.chain.from_iterable(missing_utl_type.refined_candidates))

Counter({'taxes_accrued_prepaid_and_charged_during_year_262_duration': 28809,
         'taxes_accrued_prepaid_and_charged_during_year_262_instant': 22362,
         'statement_of_income_114_duration': 21819,
         'taxes_accrued_prepaid_and_charged_during_year_totals_262_duration': 10054,
         'taxes_accrued_prepaid_and_charged_during_year_totals_262_instant': 9520,
         'summary_of_utility_plant_and_accumulated_provisions_for_depreciation_amortization_and_depletion_200_instant': 2227,
         'accumulated_deferred_income_taxes_other_property_account_282_classified_by_business_activities_274_duration': 968,
         'accumulated_deferred_income_taxes_accelerated_amortization_property_account_281_classified_by_utility_types_272_duration': 155,
         'accumulated_deferred_investment_tax_credits_account_255_total_266_duration': 100})

In [15]:
plant_in_srvce = missing_utl_type[missing_utl_type.refined_candidates.apply(lambda x: 'summary_of_utility_plant_and_accumulated_provisions_for_depreciation_amortization_and_depletion_200_instant' in x)]

In [16]:
plant_in_srvce.name.unique()

array(['utility_plant_in_service_classified',
       'utility_plant_in_service_property_under_capital_leases',
       'utility_plant_in_service_completed_construction_not_classified',
       'utility_plant_in_service_classified_and_unclassified',
       'utility_plant_held_for_future_use',
       'utility_plant_acquisition_adjustment',
       'depreciation_utility_plant_in_service',
       'amortization_of_other_utility_plant_utility_plant_in_service',
       'depreciation_amortization_and_depletion_utility_plant_in_service',
       'depreciation_utility_plant_held_for_future_use',
       'depreciation_and_amortization_utility_plant_held_for_future_use',
       'amortization_of_plant_acquisition_adjustment',
       'utility_plant_leased_to_others',
       'depreciation_utility_plant_leased_to_others',
       'depreciation_amortization_and_depletion_utility_plant_leased_to_others',
       'utility_plant_in_service_plant_purchased_or_sold',
       'utility_plant_in_service_experimental_p